# Analyzing the National Water Model with Xarray, Dask, and Coiled


This example was originally adapted from [this notebook](https://github.com/dcherian/dask-demo/blob/main/nwm-aws.ipynb), authored by Deepak Cherian, Kevin Sampson, and Matthew Rocklin.

Datasets with high spatio-temporal resolution can get very large very quickly, vastly exceeding the resources you may have on a laptop. Dask integrates with Xarray to support parallel computing and you can use Coiled to scale this to the cloud.

## The National Water Model Dataset

In this example, we'll perform a county-wise aggregation of output from the National Water Model (NWM) available on the [AWS Open Data Registry](https://registry.opendata.aws/nwm-archive/).


The National Water Model (NWM) is a highly complex hydrological modeling framework that simulates observed and forecasted streamflow across the entire continental US at a very fine spatial and temporal scale. It’s used by private and public organizations across the US to inform decision making around water management and to predict when and where flooding will occur. You can [read more from the Office of Water Prediction](https://water.noaa.gov/about/nwm).

## Problem description

We’ll calculate the mean depth to soil saturation for each US county: 

- Years: 1979-2020
- Temporal resolution: 3-hourly land surface output
- Spatial resolution: 250 m grid (~820 feet, over half a lap on a track,  2.5 soccer fields)
- 277 terabytes!

We'll use a few tools to help make this happen:
- `dask` + `coiled` process the dataset in parallel in the cloud
- `xarray` + `flox` to work with the multi-dimensional Zarr datset and aggregate to county-level means from the 250m grid.

## Start a Coiled cluster

To demonstrate calculation on a cloud-available dataset, we will use [Coiled](https://coiled.io) to set up a dask cluster in AWS `us-east-1`.

In [14]:
import coiled

cluster = coiled.Cluster(
    region="us-east-1",
    n_workers=10,
    tags={"project": "nwm"},
    scheduler_vm_types="r6in.xlarge",
    worker_vm_types="r6in.2xlarge"
)

client = cluster.get_client()

cluster.adapt(minimum=10, maximum=50)

Output()

Output()

2023-08-03 12:22:27,831 - distributed.deploy.adaptive - INFO - Adaptive scaling started: minimum=10 maximum=50


## Setup

In [15]:
%load_ext watermark

import flox  # make sure its available
import fsspec
import numpy as np
import rioxarray
import xarray as xr

xr.set_options(
    display_expand_attrs=False,
    display_expand_coords=False,
    display_expand_data=True,
)

%watermark -iv

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
rioxarray: 0.14.1
xarray   : 2023.7.0
numpy    : 1.24.4
coiled   : 0.9.0
flox     : 0.7.2
fsspec   : 2023.6.0



## Load NWM data

In [16]:
ds = xr.open_zarr(
    fsspec.get_mapper("s3://noaa-nwm-retrospective-2-1-zarr-pds/rtout.zarr", anon=True),
    consolidated=True,
    chunks={"time": 896, "x": 350, "y": 350}
)
ds

<xarray.Dataset>
Dimensions:       (time: 122479, y: 15360, x: 18432)
Coordinates: (3)
Data variables:
    crs           |S1 ...
    sfcheadsubrt  (time, y, x) float64 dask.array<chunksize=(896, 350, 350), meta=np.ndarray>
    zwattablrt    (time, y, x) float64 dask.array<chunksize=(896, 350, 350), meta=np.ndarray>
Attributes: (7)

Each field in this dataset is big!

In [17]:
ds.zwattablrt

<xarray.DataArray 'zwattablrt' (time: 122479, y: 15360, x: 18432)>
dask.array<open_dataset-zwattablrt, shape=(122479, 15360, 18432), dtype=float64, chunksize=(896, 350, 350), chunktype=numpy.ndarray>
Coordinates: (3)
Attributes: (4)

We'll subset to a single year subset for demo purposes

In [18]:
subset = ds.zwattablrt.sel(time=slice("2020-01-01", "2020-12-31"))
subset

<xarray.DataArray 'zwattablrt' (time: 2928, y: 15360, x: 18432)>
dask.array<getitem, shape=(2928, 15360, 18432), dtype=float64, chunksize=(896, 350, 350), chunktype=numpy.ndarray>
Coordinates: (3)
Attributes: (4)

## Load county raster for grouping

A raster TIFF file identifying counties by a unique integer was created separately and saved. 

We load that using [rioxarray](https://corteva.github.io/rioxarray/html/rioxarray.html)

In [19]:
import fsspec
import rioxarray

fs = fsspec.filesystem("s3", requester_pays=True)

counties = rioxarray.open_rasterio(
    fs.open("s3://nwm-250m-us-counties/Counties_on_250m_grid.tif"), chunks="auto"
).squeeze()

# remove any small floating point error in coordinate locations
_, counties_aligned = xr.align(subset, counties, join="override")

counties_aligned

<xarray.DataArray (y: 15360, x: 18432)>
dask.array<getitem, shape=(15360, 18432), dtype=int32, chunksize=(1820, 18432), chunktype=numpy.ndarray>
Coordinates: (4)
Attributes: (9)

We'll need the unique county IDs later, calculate that now.

In [20]:
county_id = np.unique(counties_aligned.data).compute()
county_id = county_id[county_id != 0]
print(f"There are {len(county_id)} counties!")

There are 3108 counties!


## GroupBy with flox

We could run the computation as

```python
subset.groupby(counties_aligned).mean()
```

This would use flox in the background.

However it would also load `counties_aligned` in to memory (an unfortunate Xarray implementation detail) which is not so bad (only a gig). To avoid egress charges, we'll instead go through `flox.xarray` which allows you to lazily groupby a dask array (here `counties_aligned`) as long as you pass in the expected group labels in `expected_groups`.

See [here](https://flox.readthedocs.io/en/latest/intro.html#with-dask) for more.

In [21]:
import flox.xarray

county_mean = flox.xarray.xarray_reduce(
    subset,
    counties_aligned.rename("county"),
    func="mean",
    expected_groups=(county_id,),
)

county_mean

<xarray.DataArray 'zwattablrt' (time: 2928, county: 3108)>
dask.array<groupby_nanmean, shape=(2928, 3108), dtype=float64, chunksize=(896, 3108), chunktype=numpy.ndarray>
Coordinates: (4)
Attributes: (4)

In [22]:
county_mean.load()

<xarray.DataArray 'zwattablrt' (time: 2928, county: 3108)>
array([[1.6148426 , 1.77337928, 1.64958168, ..., 1.96216723, 1.88654191,
        1.66249746],
       [1.61630079, 1.7736926 , 1.65047253, ..., 1.9620833 , 1.88661932,
        1.66218315],
       [1.6158894 , 1.77379078, 1.65159624, ..., 1.96205491, 1.88628079,
        1.66221987],
       ...,
       [1.65066191, 1.73475125, 1.52970677, ..., 1.97500157, 1.96858928,
        1.95776549],
       [1.65242966, 1.73532446, 1.53078175, ..., 1.97483371, 1.96860777,
        1.95795127],
       [1.652059  , 1.73476713, 1.52733207, ..., 1.97489542, 1.96873255,
        1.95791995]])
Coordinates: (4)
Attributes: (4)

The computation proceeds very nicely. 

We don't anticipate trouble scaling this computation up to the full dataset.

## Cleanup

In [23]:
cluster.shutdown()

2023-08-03 12:36:51,023 - distributed.deploy.adaptive_core - INFO - Adaptive stop
2023-08-03 12:36:51,583 - distributed.deploy.adaptive_core - INFO - Adaptive stop


## Visualize yearly mean

Read county shapefile, combo of state FIPS code and county FIPS code as multi-index

In [35]:
import geopandas as gpd
import hvplot.pandas

counties = gpd.read_file(
    "https://www2.census.gov/geo/tiger/GENZ2022/shp/cb_2022_us_county_20m.zip"
).to_crs("EPSG:3395")
counties["STATEFP"] = counties.STATEFP.astype(int)
counties["COUNTYFP"] = counties.COUNTYFP.astype(int)
continental = counties[~counties["STATEFP"].isin([2, 15, 72])].set_index(["STATEFP", "COUNTYFP"])

Interpret `county` as combo of state FIPS code and county FIPS code. Set multi-index

In [28]:
yearly_mean = county_mean.mean("time")
yearly_mean.coords["STATEFP"] = (yearly_mean.county // 1000).astype(int)
yearly_mean.coords["COUNTYFP"] = np.mod(yearly_mean.county, 1000).astype(int)
yearly_mean = yearly_mean.drop_vars("county").set_index(county=["STATEFP", "COUNTYFP"])
yearly_mean

<xarray.DataArray 'zwattablrt' (county: 3108)>
array([1.67101811, 1.5468516 , 1.56150642, ..., 1.94395235, 1.92369862,
       1.89985533])
Coordinates: (5)

Join

In [31]:
continental["zwattablrt"] = yearly_mean.to_dataframe()["zwattablrt"]

Plot

In [34]:
continental.hvplot(
    c="zwattablrt",
    cmap='turbo_r',
    title="Mean Depth to Soil Saturation in 2020 by US County (meters)",
    xaxis=None,
    yaxis=None
)

:Polygons   [x,y]   (zwattablrt)